In [1]:
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request, register_request
from quantumnet.objects import Request, time
import asyncio
path = "resultados/proativo/sync.csv"
rede = Network()
rede.n_initial_eprs = 10
rede.n_initial_qubits = 10
controlador = Controller(rede)
controlador.default_ttl = 100
row, col = 3, 4
rede.set_ready_topology("Grade", row, col)
# Criação de requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 100
requests = list(generate_random_request(len(rede.hosts)-1, fmin_range, neprs_range) for i in range(num_requests))

TypeError: LinkLayer.__init__() takes 2 positional arguments but 3 were given

In [ ]:
hosts = rede.hosts
for alice in hosts:
    print(f"Adicionando regras para {alice}")
    for bob in hosts:
        controlador.add_match_route_rule_in_host_proactive(alice, bob, (0.5, 0.6), 5)
        controlador.add_match_route_rule_in_host_proactive(alice, bob, (0.8, 0.9), 5)
    # Mostra as tabelas
    #rede.get_host(alice).draw_flow_table()

In [3]:
def update_rules(host, time_for_update):
    for match in host.flow_table.copy():
        rule = host.flow_table[match][1]
        
        for t in range(time_for_update):
            rule.increment_age()
            
        if rule.ttl <= rule.age:
            if rule.hit_count == 0:
                host.flow_table.pop(match)
                print(f"[Time {time.get_current_time()}] Regra {rule} expirada em {host}")
            else:
                print(f"Regra reutilizada {rule} em {host}")
                rule.hit_count = 0
                rule.age = rule.ttl/2

def update_time(time_for_update):
    # Atualiza o tempo
    for t in range(time_for_update):
        time.increment()
        if time.get_current_time() % 10 == 0:
            rede.refresh_resources(num_qubits=10, num_eprs=10)
            print(f"[Time {time.get_current_time()}] Recursos atualizados")
            
    # Atualiza as regras
    for host in rede.hosts:
        update_rules(rede.get_host(host), time_for_update)

In [ ]:
# Parte adicional para simulação com timeslot
def simulate_requests_with_timeslots(rede, requests, controlador, time):
    """
    Simula o processamento de requests com a introdução de timeslots.
    - Demora 1 time-slot se a regra existe.
    - Demora 3 time-slots se uma nova regra precisa ser adicionada.
    """
    for request in requests:
        print(f"[Time {time.get_current_time()}] Processando requisição {request}...")
        alice = rede.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = time.get_current_time()
            update_time(3)
            controlador.add_match_route_rule_in_host_reactive(request)
            rule = alice.find_rule_by_request(request)
            controlador.run_rule(rule[1])
            request.endtime = time.get_current_time() 
            # Registra no CSV como um novo registro
            register_request(request, "Novo Registro", path)
        else:  # Caso já exista a regra
            request.starttime = time.get_current_time()
            update_time(1)
            controlador.run_rule(rule[1])
            request.endtime = time.get_current_time()
            # Registra no CSV como já registrado
            register_request(request, "Já Registrado", path)
        
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

simulate_requests_with_timeslots(rede, requests, controlador, time)